# Performing a convergence study

This example shows how to perform a convergence study to find an appropriate discretisation parameters for the Brillouin zone (`kgrid`) and kinetic energy cutoff (`Ecut`), such that the simulation results are converged to a desired accuracy tolerance.

Such a **convergence study** is generally performed by starting with a reasonalbe base line value for `kgrid` and `Ecut` and then increasing these parameters (i.e. using finer discretisations) until a desired property (such as the energy) changes less than the tolerance.
This procedure must be performed for each discretisation parameter. Beyond the `Ecut` and the `kgrid` also convergence in the smearing temperature or other numerical parameters should be checked. For simplicity we will neglect this aspect in this example and concentrate on `Ecut` and `kgrid`. Moreover we will restrict ourselves to using the same number of $k$-points in each dimension of the Brillouin zone.

As the objective of this study we consider bulk platinum. For running the SCF conveniently we define a function:

In [ ]:
using DFTK
using Unitful
using UnitfulAtomic
using LinearAlgebra
using Statistics

function run_scf(; a=5.0, Ecut, nkpt, tol)
    atoms    = [ElementPsp(:Pt, psp = load_psp("hgh/lda/Pt-q10"))]
    position = [zeros(3)]
    lattice  = a * Matrix(I, 3, 3)

    model  = model_LDA(lattice, atoms, position; temperature=1e-3)
    basis  = PlaneWaveBasis(model; Ecut, kgrid=(nkpt, nkpt, nkpt))
    println("Ecut = $Ecut, nkpt = $nkpt")
    self_consistent_field(basis; tol)
end

Moreover we define some parameters:

In [ ]:
tol   = 1e-3   # Tolerance to which we target to converge
nkpts = 1:18   # K-point range checked for convergence
Ecuts = (300:50:1000)u"eV";  # Energy cutoff range checked for convergence

As the first step we converge in the number of kpoints employed in each dimension of the Brillouin zone.

In [ ]:
Ecut_kconv = mean(Ecuts)
energies = [run_scf(; nkpt, tol=tol/50, Ecut=Ecut_kconv).energies.total for nkpt in nkpts]
errors = abs.(energies[1:end-1] .- energies[end])
iconv = findfirst(errors .< tol)
nkpt_conv = nkpts[iconv]

... and plot the obtained convergence:

In [ ]:
using Plots
plot(nkpts[1:end-1], errors, dpi=300, lw=3, m=:o, yaxis=:log, label="",
     xlabel="k-grid", ylabel="energy absolute error")

We continue to do the convergence in Ecut using the suggested k-point grid.

In [ ]:
energies = [run_scf(; nkpt=nkpt_conv, tol=tol/10, Ecut).energies.total for Ecut in Ecuts]
errors = abs.(energies[1:end-1] .- energies[end])
iconv = findfirst(errors .< tol)
Ecut_conv = Ecuts[iconv]

... and plot it:

In [ ]:
plot(ustrip.(Ecuts[1:end-1]), errors, dpi=300, lw=3, m=:o, yaxis=:log, label="",
     xlabel="Ecut (eV)", ylabel="energy absolute error")